In [2]:
import pydocparser

api_key = 'a97eb5d696857fea8bfa9e57fb9994017a3bb7da'
parser_id = 'paper_parser'

parser = pydocparser.Parser()
parser.login(api_key)
result = parser.ping()
print(result)

pong


In [3]:
parsers = parser.list_parsers()
parsers

[{'id': 'ucciynzmvnap', 'label': 'paper_parser'},
 {'id': 'lnhiyzpmkoyj', 'label': 'pdf_parser'}]

In [4]:
id = parser.upload_file_by_path("data_samples/electricity-theft-detection-using-machine-learning-IJERTCONV10IS04024.pdf", "paper_parser") 

In [5]:
data = parser.get_one_result("paper_parser", id)

In [6]:

data

[{'id': 'd6b3724a8b804670377019f0759a95b7',
  'document_id': '7c545cd9ab2fb771be6f5ab82579833b',
  'remote_id': '',
  'file_name': 'electricity-theft-detection-using-machine-learning-IJERTCONV10IS04024.pdf',
  'media_link': 'https://api.docparser.com/v1/document/media/wXoTq27bjIzsrCCIsm2KHvWS6TRyuZkMPrqyumNlrr13alakMyntbBQaYV03A5jRTBjON-ogp_eW5dqfJw8rvR5rh-dkdqzHAsD8KyldkEk',
  'media_link_original': 'https://api.docparser.com/v1/document/media/wXoTq27bjIzsrCCIsm2KHvWS6TRyuZkMPrqyumNlrr13alakMyntbBQaYV03A5jRTBjON-ogp_eW5dqfJw8rvR5rh-dkdqzHAsD8KyldkEk/original',
  'media_link_data': 'https://api.docparser.com/v1/document/media/wXoTq27bjIzsrCCIsm2KHvWS6TRyuZkMPrqyumNlrr13alakMyntbBQaYV03A5jRTBjON-ogp_eW5dqfJw8rvR5rh-dkdqzHAsD8KyldkEk/data',
  'page_count': 4,
  'uploaded_at': '2024-01-22T15:55:25+00:00',
  'processed_at': '2024-01-22T15:55:27+00:00',
  'uploaded_at_utc': '2024-01-22T15:55:25+00:00',
  'uploaded_at_user': '2024-01-22T07:55:25+00:00',
  'processed_at_utc': '2024-01-22T15:5

In [7]:
keywords = data[0]['keywords']
keywords

"-Energy Meter, Convolutional Neural Network ,                   additionally affect the decisions made through the SO\n   Advanced Meter Infrastructure, System Operators, System                  misleading behaviour does not simplest cause economic\n   Grid                                                                     regarding grid management, which can also reason the\n                     I. INTRODUCTION                                        instability of the grid or blackout in intense cases . energy\n Energy meter, or kilowatt-hour meter is a tool that measures               theft is a critical trouble inside the existing power grid that\nthe amount of electric energy consumed via a house, a                       causes hefty monetary losses. We deal with in this paper is a\ncommercial enterprise,or an electrically powered tool                       way to permit the to be able to display load, compute\n.Electric utilities use power meters installed at purchaser          

In [8]:
abstract = data[0]['abstract']
abstract

"- Nowadays energy is very crucial , so we want to make             each billing length.The loss of power in electricity\nthe proper use of energy sources particularly electricity                   transmission and distribution is an essential problem\nutilization. As one of the foremost elements of the nontechnical            confronted through electricity groups all over the\nlosses (NTLs) in distribution networks, the energy theft reasons            international. The energy losses are commonly categorised\ngreat harm to strength grids, which influences power supply\ngreat and decreases working profits. In advanced metering\n                                                                            into technical losses (TLs) and nontechnical losses (NTLs).\ninfrastructure (AMI), smart meters (SMs) are hooked up on the               The TL is inherent to the transportation of strength, that is\ncustomer aspect to ship excellent- grained energy consumption               resulting fr

In [10]:
references = data[0]['references']
references

"[1] D. Yao, M. Wen, X. Liang, Z. Fu, K. Zhang, and B. Yang, “Energy\n                                                                              theft detection with energy privacy preservation in the smart grid,”\n                                                                              IEEE Internet of Things Journal, vol. 6, no. 5, pp. 7659-7669, Oct.\n                                                                              2019\n                                                                        [2]   P. Jokar, N. Arianpoo, and V. C. M. Leung, “Electricity theft\n                                                                              detection in AMI using customers consumption patterns,” IEEE\n                                                                              Transactions on Smart Grid, vol. 7, no. 1, pp. 216-226, Jan. 2016\n                                                                        [3]   M. Nabil, M. Ismail, M. M. E. A. Mahmoud, W. Al

In [2]:
import PyPDF2

def split_pdf_vertically(input_pdf_path, output_pdf_path_prefix):
    with open(input_pdf_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)

        for page_num in range(len(pdf_reader.pages)):
            pdf_writer = PyPDF2.PdfWriter()
            pdf_writer.addPage(pdf_reader.getPage(page_num))

            output_pdf_path = f"{output_pdf_path_prefix}_page{page_num + 1}.pdf"

            with open(output_pdf_path, 'wb') as output_pdf_file:
                pdf_writer.write(output_pdf_file)

if __name__ == "__main__":
    input_pdf_path = "data_samples/electricity-theft-detection-using-machine-learning-IJERTCONV10IS04024.pdf"
    output_pdf_path_prefix = "extracted_text/output_split"

    split_pdf_vertically(input_pdf_path, output_pdf_path_prefix)


DeprecationError: reader.numPages is deprecated and was removed in PyPDF2 3.0.0. Use len(reader.pages) instead.